In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator (rescale = 1./255, shear_range = 0.2, zoom_range = [.99, 1.01], brightness_range= [0.8, 1.2], data_format= "channels_last", fill_mode="constant", horizontal_flip = True)
test_datagen = ImageDataGenerator (rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/naan mudhalvan/train_data',target_size = (180, 180),batch_size = 64,class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/naan mudhalvan/test_data',target_size = (180, 180),batch_size = 64,class_mode = 'categorical')

Found 1200 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


In [ ]:
VGG19 = VGG19(input_shape=[180,180]+ [3], weights='imagenet',include_top=False)
for layer in VGG19.layers: layer.trainable = False
x = Flatten() (VGG19.output)

80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
#Adding Dense Layers
prediction = Dense(5, activation = 'softmax')(x)

In [ ]:
model = Model(inputs = VGG19.input, outputs=prediction)


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
#Train The Model
r = model.fit(
     training_set,
     validation_data=test_set,
     epochs=50,
     steps_per_epoch=len(training_set),
     validation_steps=len(test_set)
)

Epoch 1/50
19/19 [==============================] - 120s 7s/step - loss: 0.6860 - accuracy: 0.7658 - val_loss: 0.5960 - val_accuracy: 0.7967
Epoch 2/50
19/19 [==============================] - 26s 1s/step - loss: 0.4650 - accuracy: 0.8483 - val_loss: 0.5875 - val_accuracy: 0.7967
Epoch 3/50
19/19 [==============================] - 25s 1s/step - loss: 0.3742 - accuracy: 0.8933 - val_loss: 0.4781 - val_accuracy: 0.8333
Epoch 4/50
19/19 [==============================] - 26s 1s/step - loss: 0.3236 - accuracy: 0.9117 - val_loss: 0.4254 - val_accuracy: 0.8700
Epoch 5/50
19/19 [==============================] - 26s 1s/step - loss: 0.2971 - accuracy: 0.9225 - val_loss: 0.4941 - val_accuracy: 0.8367
Epoch 6/50
19/19 [==============================] - 26s 1s/step - loss: 0.2707 - accuracy: 0.9325 - val_loss: 0.4270 - val_accuracy: 0.8567
Epoch 7/50
19/19 [==============================] - 25s 1s/step - loss: 0.2337 - accuracy: 0.9350 - val_loss: 0.3874 - val_accuracy: 0.8733
Epoch 8/50
19/19 [=

In [ ]:
loss, accuracy = model.evaluate(test_set,
                                steps=11,
                                verbose=2,
                                use_multiprocessing=True,
                                workers=2)
print(f'Model performance on test images"\nAccuracy = {accuracy}\nloss={loss}')

11/11 - 6s - loss: 0.3503 - accuracy: 0.8946 - 6s/epoch - 537ms/step
Model performance on test images"
Accuracy = 0.8945783376693726
loss=0.3502841293811798


In [ ]:
model.save('model.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model=load_model("/content/model.h5")

In [ ]:
img = image.load_img(r"/content/drive/MyDrive/naan mudhalvan/test_data/shine/shine202.jpg", target_size=(180, 180)) # loading the image
X = image.img_to_array(img) # convert the image to an array
X = np.expand_dims(X, axis=0) # expand the dimensions of the array
preds = model.predict(X)
pred = np.argmax(preds, axis=1)
index = ['cloudy', 'foggy', 'rainy', 'shine', 'sunrise']
result = str(index[pred[0]])
result


1/1 [==============================] - 1s 832ms/step


'shine'

In [ ]:
!tar -zcvf modelWeights.tgz model.h5

model.h5
